# ♟️ Chess Move Montage: AI-Powered Highlight Reel
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/editor/creative/chess_montage.ipynb)

Turn a long chess match into a **punchy highlight reel** — automatically!

## The Pipeline

1. **Upload** chess video + background music
2. **Index scenes** with simple move detection ("Player Moved" / "No Move")
3. **Extract timestamps** — LLM returns just start times as JSON
4. **Build timeline** — 5-second clips with transitions, filters, and music

No manual editing. No frame-by-frame scrubbing. Just code → montage.

---

## Step 1: Setup

In [ ]:
%pip -q install git+https://github.com/video-db/videodb-python.git@ankit/add-videodb-editor

  Preparing metadata (setup.py) ... done


### Connect to VideoDB

In [ ]:
import videodb
import os
from getpass import getpass

api_key = getpass("Enter your VideoDB API Key: ")
os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()
coll = conn.get_collection()
print("✓ Connected!")

Enter your VideoDB API Key: ··········
✓ Connected!


---

## Step 2: Upload Assets

We need two assets:
- **Chess video** — the full match
- **Background music** — energetic audio for the montage

In [ ]:
from videodb import MediaType

# Upload chess video
chess_video = coll.upload(url="https://www.youtube.com/watch?v=dhDe-RcoyAU")
print(f"✓ Chess video uploaded: {chess_video.id}")

# chess_video = coll.get_video("your_video_id")

✓ Chess video uploaded: m-z-019b44ba-80cb-7d70-9bd8-3c480e581777


In [ ]:
# Upload background music
bg_music = coll.upload(
    url="https://www.youtube.com/watch?v=S19UcWdOA-I",
    media_type=MediaType.audio
)
print(f"✓ Background music uploaded: {bg_music.id}")

# bg_music = coll.get_audio("your_audio_id")

✓ Background music uploaded: a-z-019b44bb-95d8-72e3-a62d-dc8c369d0756


### Preview the chess video

In [ ]:
from videodb import play_stream
play_stream(chess_video.generate_stream())

---

## Step 3: Index Scenes with Simple Move Detection

We index the video with a **strict binary prompt**: each scene is either "Player Moved" or "No Move".

No complex move identification — just detecting WHEN a move happens.

In [ ]:
from videodb import SceneExtractionType

print("Indexing scenes for move detection...")

moves_index_id = chess_video.index_scenes(
    extraction_type=SceneExtractionType.time_based,
    extraction_config={"time": 8, "frame_count": 5},
    prompt="""Look at this chess scene and focus on the chess board. Your task is to detect when pieces are moved.

Respond with ONLY one of these two keywords:
- "Player Moved" — if a chess piece was moved
- "No Move" — if no move occurred (same position, paused, talking, etc.)

Be strict. Only say "Player Moved" if you clearly see a chess piece moved.""",
    name="Chess_Move_Detection"
)

print(f"✓ Indexing complete. Index ID: {moves_index_id}")

✓ Indexing complete. Index ID: 5ef1107be779d1c0


### Viewing the scene index just  created


In [ ]:
import json
moves_scenes = chess_video.get_scene_index(moves_index_id)
print(json.dumps(moves_scenes, indent=2))

[
  {
    "description": "No Move",
    "end": 8.0,
    "metadata": {},
    "scene_metadata": {},
    "start": 0.0
  },
  {
    "description": "No Move",
    "end": 16.0,
    "metadata": {},
    "scene_metadata": {},
    "start": 8.0
  },
  {
    "description": "No Move\nPlayer Moved\nNo Move\nNo Move",
    "end": 24.0,
    "metadata": {},
    "scene_metadata": {},
    "start": 16.0
  },
  {
    "description": "No Move",
    "end": 32.0,
    "metadata": {},
    "scene_metadata": {},
    "start": 24.0
  },
  {
    "description": "Player Moved\nPlayer Moved\nPlayer Moved\nPlayer Moved",
    "end": 40.0,
    "metadata": {},
    "scene_metadata": {},
    "start": 32.0
  },
  {
    "description": "Player Moved",
    "end": 48.0,
    "metadata": {},
    "scene_metadata": {},
    "start": 40.0
  },
  {
    "description": "No Move\nNo Move\nPlayer Moved\nPlayer Moved\nNo Move",
    "end": 56.0,
    "metadata": {},
    "scene_metadata": {},
    "start": 48.0
  },
  {
    "description": "Player 

---

## Step 4: Extract Move Timestamps with LLM

Now we feed the scene index to the LLM and ask for **just the start timestamps** of every detected move.

The output is a simple JSON array: `[0, 8, 24, 32, ...]` — nothing else.

In [ ]:
prompt = f"""Analyze the scene descriptions from this chess video.

Find EVERY scene where the description says "Player Moved".

Return a JSON array containing ONLY the start timestamps (in seconds) of those scenes.

Example output format:
[0, 8, 16, 24, 40, 48]

Rules:
- Return ONLY the JSON array, nothing else
- No descriptions, no explanations, just timestamps

Moves Index : "{moves_scenes}"
"""

print("Extracting move timestamps...")
response = coll.generate_text(
    prompt=prompt,
    response_type="json",
    model_name="pro"
)

print("✓ LLM analysis complete!")

Extracting move timestamps...
✓ LLM analysis complete!


### Parse the timestamps

In [ ]:
import json

# Parse timestamps from LLM response
timestamps = response.get('output', response)
if isinstance(timestamps, str):
    timestamps = json.loads(timestamps)

print(f"✓ Found {len(timestamps)} moves")
print(f"Timestamps: {timestamps}")

✓ Found 69 moves
Timestamps: [16.0, 32.0, 40.0, 48.0, 56.0, 64.0, 72.0, 96.0, 104.0, 112.0, 136.0, 144.0, 168.0, 176.0, 184.0, 200.0, 208.0, 216.0, 240.0, 248.0, 256.0, 264.0, 272.0, 288.0, 296.0, 300.0, 316.0, 324.0, 340.0, 348.0, 356.0, 364.0, 380.0, 388.0, 404.0, 412.0, 420.0, 428.0, 444.0, 452.0, 468.0, 476.0, 484.0, 492.0, 508.0, 516.0, 540.0, 548.0, 556.0, 564.0, 572.0, 580.0, 596.0, 600.0, 608.0, 616.0, 648.0, 656.0, 664.0, 680.0, 696.0, 728.0, 752.0, 776.0, 792.0, 816.0, 832.0, 864.0, 896.0]


---

## Step 5: Build the Timeline

Now we assemble the montage step-by-step:
1. Create timeline
2. Add video clips (8 seconds each)
3. Apply filters and transitions
4. Add background music

### 5.1 Initialize Timeline with Intro Text

In [ ]:
from videodb.editor import (
    Timeline, Track, Clip, VideoAsset, AudioAsset,
    Filter, Transition, TextAsset, Font, Background,
    Alignment, HorizontalAlignment, VerticalAlignment, TextAlignment,
    Position, Offset
)

timeline = Timeline(conn)
timeline.background = "#000000"

# Create intro text: "Let the Match Begin"
intro_text = TextAsset(
    text="Let the Match Begin",
    font=Font(family="Clear Sans", size=56, color="#FFFFFF"),
    background=Background(
        width=600,
        height=120,
        color="#000000",
        border_width=2.0,
        text_alignment=TextAlignment.center
    ),
    alignment=Alignment(
        horizontal=HorizontalAlignment.center,
        vertical=VerticalAlignment.center
    ),
)

intro_clip = Clip(
    asset=intro_text,
    duration=3,
    transition=Transition(in_="fade", out="fade", duration=0.5)
)

intro_track = Track()
intro_track.add_clip(0, intro_clip)
timeline.add_track(intro_track)

print("✓ Timeline initialized with 3-second intro")

# Preview the intro
stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

✓ Timeline initialized with 3-second intro
Stream URL: https://play.videodb.io/v1/5a4a8027-b0d1-4570-9169-9aa52ad99f56.m3u8


### 5.2 Add Video Clips

Each move gets an **8-second clip** with:
- **Contrast filter** — for a dramatic look
- **Fade transitions** — smooth in/out between clips

In [ ]:
CLIP_DURATION = 5  # seconds per clip
TARGET_CLIPS = 10  # approximately how many clips we want

# Sample timestamps evenly across the full list
total_detected = len(timestamps)
step = max(1, total_detected // TARGET_CLIPS)  # e.g., 69 // 10 = 6, so pick every 6th
sampled_timestamps = timestamps[::step][:TARGET_CLIPS]  # Take every Nth, limit to TARGET_CLIPS

print(f"Detected {total_detected} moves, sampling {len(sampled_timestamps)} clips (every {step}th)")
print(f"Sampled timestamps: {sampled_timestamps}")

video_track = Track()
timeline_position = 3

for i, start_time in enumerate(sampled_timestamps):
    clip = Clip(
        asset=VideoAsset(
            id=chess_video.id,
            start=start_time-1,
            volume=0  # Muting the original audio
        ),
        duration=CLIP_DURATION,
        filter=Filter.contrast,
        transition=Transition(in_="fade", out="fade", duration=1)
    )

    video_track.add_clip(timeline_position, clip)
    timeline_position += CLIP_DURATION
    print(f"Clip {i+1}: source {start_time}s → timeline {timeline_position - CLIP_DURATION}s")

timeline.add_track(video_track)

total_duration = len(sampled_timestamps) * CLIP_DURATION + 3
print(f"\n✓ Added {len(sampled_timestamps)} clips to timeline")
print(f"✓ Total montage duration: {total_duration}s")

stream_url = timeline.generate_stream()
play_stream(stream_url)

Detected 69 moves, sampling 10 clips (every 6th)
Sampled timestamps: [16.0, 72.0, 168.0, 240.0, 296.0, 356.0, 420.0, 484.0, 556.0, 608.0]
Clip 1: source 16.0s → timeline 3s
Clip 2: source 72.0s → timeline 8s
Clip 3: source 168.0s → timeline 13s
Clip 4: source 240.0s → timeline 18s
Clip 5: source 296.0s → timeline 23s
Clip 6: source 356.0s → timeline 28s
Clip 7: source 420.0s → timeline 33s
Clip 8: source 484.0s → timeline 38s
Clip 9: source 556.0s → timeline 43s
Clip 10: source 608.0s → timeline 48s

✓ Added 10 clips to timeline
✓ Total montage duration: 53s


### 5.3 Add Background Music

In [ ]:
# Calculate total montage duration
music_duration = total_duration

# Add music track
music_clip = Clip(
    asset=AudioAsset(
        id=bg_music.id,
        start=0,
        volume=0.7
    ),
    duration=total_duration
)

audio_track = Track()
audio_track.add_clip(0, music_clip)
timeline.add_track(audio_track)

print(f"✓ Added {total_duration}s of background music")

stream_url = timeline.generate_stream()
play_stream(stream_url)

✓ Added 53s of background music


---

## 🎉 What We Built

**Input:** Long chess match video + background music

**Output:** Punchy highlight reel of every move

### The Pipeline

| Step | What Happens |
|------|--------------|
| **Upload** | Chess video + music to VideoDB |
| **Index** | Binary detection: "Player Moved" / "No Move" |
| **Extract** | LLM returns just start timestamps as JSON array |
| **Build** | 5-second clips + contrast filter + fade transitions + music |
| **Generate** | Final montage stream |

### Key Techniques

- **Simple prompting** — binary keywords give cleaner LLM input
- **Hardcoded duration** — no LLM complexity for clip length
- **Single track iteration** — just loop over timestamps list
- **Filter + Transition** — instant polish with one-liners